In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_1월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 13120/13120 [40:55<00:00,  5.34it/s] 


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/gon0033/222636070981,부산 기장 카페 <코랄라니> / 크기에 압도당하고 접근성에...,2022.01.31.,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다\n사실은 밀려있는 그릿비에게 밀...,0,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다 사실은 밀려있는 그릿비에게 밀려...,"[(방문한지 4달이 조금 안된시점에 포스팅을 하게되었다, 0, 28, None, [..."
1,https://blog.naver.com/wingwing28/222636070507,중국에서 어떻게 설날을 보낼까?｜만두 빚기｜어마어마한...,2022.01.31.,안녕하세요 벨라예요\n여러분 설날이네요~ 새해가 정말 시작됐군요 광광 ㅠㅠㅠㅠㅠㅠ ...,1,안녕하세요 벨라예요 여러분 설날이네요 새해가 정말 시작됐군요 광광 ㅠ 마지막 20대...,"[(안녕하세요 벨라예요, 0, 10, None, []), (여러분 설날이네요, 11..."
2,https://blog.naver.com/ossong0302/222636070130,"전시:: 카유보트, 향기를 만나다. 파주에서 만난 프랑스 작가",2022.01.31.,향기와 함께하는 전시가 있다는 말에\n냉큼 가보기로 했다.\n파주 헤이리마을\n작은...,2,향기와 함께하는 전시가 있다는 말에 냉큼 가보기로 했다 파주 헤이리마을 작은 전시관...,"[(향기와 함께하는 전시가 있다는 말에 냉큼 가보기로 했다, 0, 30, None,..."
3,https://blog.naver.com/yellowwish/222636068312,국립박물관 굿즈 나전합 자개 스마트톡과 금속배지,2022.01.31.,"나전합의 매력 그대로\n자개 스마트톡, 금속배지\n어린 시절에 용돈을 받으면 어디에...",3,나전합의 매력 그대로 자개 스마트톡 금속배지 어린 시절에 용돈을 받으면 어디에 쓰셨...,[(나전합의 매력 그대로 자개 스마트톡 금속배지 어린 시절에 용돈을 받으면 어디에 ...
4,https://blog.naver.com/sunjin8296/222636066942,설맞이 수정과 만들기_네이처리얼 시나몬스틱으로 만들었어요!,2022.01.31.,저희 시댁은 차례나 명절음식 따로 없이\n명절에 모이면 회를 떠다가 초밥을 만들어 ...,4,저희 시댁은 차례나 명절음식 따로 없이 명절에 모이면 회를 떠다가 초밥을 만들어 먹...,[(저희 시댁은 차례나 명절음식 따로 없이 명절에 모이면 회를 떠다가 초밥을 만들어...
...,...,...,...,...,...,...,...
13115,https://blog.naver.com/kimys413/222607967751,건지산 숲속 놀이터 베짱이숲!!,2022.01.01.,전주 체련공원과 동물원 사이 건지산에 베짱이숲이라는 숲속 놀이터가 있는데용~~~\n...,13115,전주 체련공원과 동물원 사이 건지산에 베짱이숲이라는 숲속 놀이터가 있는데용 블로그 ...,[(전주 체련공원과 동물원 사이 건지산에 베짱이숲이라는 숲속 놀이터가 있는데용 블로...
13116,https://blog.naver.com/01196292191/222609254076,중국의 뱀파이어 강시...,2022.01.01.,안녕하세요?\n깜찍이헐크입니다..\n자 오늘은 중국의 뱀파이어 강시에 대해 포스팅 ...,13116,안녕하세요 깜찍이헐크입니다 자 오늘은 중국의 뱀파이어 강시에 대해 포스팅 해보겠습니...,"[(안녕하세요 깜찍이헐크입니다, 0, 14, None, []), (자 오늘은 중국의..."
13117,https://blog.naver.com/rho8911/222609258593,제주 숨도 석부작박물관 힐링 명소 추천,2022.01.01.,주부라니 입니다 :)\n새해 복 많이 받으세요\n소원은 잘 빌고 오셨나요 ?\n올 ...,13117,주부라니 입니다 새해 복 많이 받으세요 소원은 잘 빌고 오셨나요 올 해도 건강하시고...,"[(주부라니 입니다, 0, 8, None, []), (새해 복 많이 받으세요, 9,..."
13118,https://blog.naver.com/sheshow/222603519035,"역사를 찾아서 497편 신악(新樂)을 창제하다-세종, 신악을...",2022.01.01.,blog\n조선제4대세종 177개의 글\n조선제4대세종\n목록닫기\n조선제4대세종 ...,13118,blog 조선제4대세종 177개의 글 조선제4대세종 목록닫기 조선제4대세종 글 목록...,[(blog 조선제4대세종 177개의 글 조선제4대세종 목록닫기 조선제4대세종 글 ...


In [5]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [9]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_1월_전처리.csv", index=False, encoding='UTF-8')

In [6]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_1월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [7]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_1월_전처리.csv", index=False, encoding='UTF-8')
df2

[8211]


In [7]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../나무문화/2022/나무문화_본문_1월_전처리.csv", index=False, encoding='UTF-8')